In [ ]:
import sys
sys.path.insert(0,'../')
sys.path

In [ ]:
import obsidian
print(f'obsidian version: ' + obsidian.__version__)

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

## Introduction

In this tutorial, we will see how to use _obsidian_ for multi-output optimization. To demonstrate the versatility of the approach, we will seek to maximize one response while minimizing the other.

$$\underset{X}{argmax}  HV\left(+f\left(y_1\right) -f\left(y_2\right)\right)$$

Furthermore, we will apply a linear constraint on the input variables; requiring that the $X_1 + X_2 \leq 6 $.

## Set up parameter space and initialize a design

In [ ]:
from obsidian import Campaign, Target, ParamSpace, BayesianOptimizer
from obsidian.parameters import Param_Continuous

In [ ]:
params = [
    Param_Continuous('X1', 0, 10),
    Param_Continuous('X2', 0, 10),
    ]

X_space = ParamSpace(params)
target = [
    Target('Response 1', aim='max'),
    Target('Response 2', aim='min')
]
campaign = Campaign(X_space, target, seed=0)
X0 = campaign.designer.initialize(4, 'LHS')

X0

## Collect results (e.g. from a simulation)

In [ ]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import branin_currin

simulator = Simulator(X_space, branin_currin, name='Response', eps=0.05)
y0 = simulator.simulate(X0)
Z0 = pd.concat([X0, y0], axis=1)

campaign.add_data(Z0)
campaign.data

### Fit an optimizer and visualize results

In [ ]:
campaign.fit()

In [ ]:
from obsidian.plotting import surface_plot, optim_progress

In [ ]:
surface_plot(campaign.optimizer)

## Optimize new experiment suggestions

In [ ]:
from obsidian.constraints import Linear_Constraint

_Note:_ It is a good idea to balance a set of acquisition functions with those that prefer design-space exploration. This helps to ensure that the optimizer is not severely misled by deficiencies in the dataset, particularly for small data. It also helps to ascertain a global optimum.

A simple choice is __Space Filling (SF)__ although __Negative Integrated Posterior Variance (NIPV)__ is available for single-output optimizations; and there are various other acquisiiton functions whose hyperparameters can be tuned to manage the "explore-exploit" balance.

In [ ]:
X_suggest, eval_suggest = campaign.optimizer.suggest(acquisition = [{'NEHVI':{'ref_point':[-350, -20]}}, 'SF'],
                                                     # X1 + X2 <= 6, written as -X1 - X2 >= -6
                                                     ineq_constraints = [Linear_Constraint(X_space, ind=[0,1], weights=[-1,-1], rhs=-6, equality=True)])

In [ ]:
pd.concat([X_suggest, eval_suggest], axis=1)

## Collect data at new suggestions

In [ ]:
y_iter1 = pd.DataFrame(simulator.simulate(X_suggest))
Z_iter1 = pd.concat([X_suggest, y_iter1, eval_suggest], axis=1)
campaign.add_data(Z_iter1)
campaign.data.tail()

## Repeat as desired

In [ ]:
for iter in range(5):
    campaign.fit()
    X_suggest, eval_suggest = campaign.optimizer.suggest(acquisition = [{'NEHVI':{'ref_point':[-350, -20]}}, 'SF'],
                                                         ineq_constraints = [Linear_Constraint(X_space, ind=[0,1], weights=[-1,-1], rhs=-6, equality=True)])
    y_iter = pd.DataFrame(simulator.simulate(X_suggest))
    Z_iter = pd.concat([X_suggest, y_iter, eval_suggest], axis=1)
    campaign.add_data(Z_iter)

In [ ]:
optim_progress(campaign)

In [ ]:
surface_plot(campaign.optimizer, response_id = 0)

In [ ]:
surface_plot(campaign.optimizer, response_id = 1)